In [1]:
import os
import glob

import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda122.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda122.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/user/sabiyyuhakim/oauth_callback')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('["access'), PosixPath('servers!server=sabiyyuhakim/", "access'), PosixPath('servers!user=sabiyyuhakim"]')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//127.0.0.1'), PosixPath('8081/hub/api'), PosixPath('http')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8081/hu

In [2]:
ROOT_DATASET_PATH = Path('./cifake/')
TRAIN_DATASET_PATH = ROOT_DATASET_PATH / 'train'
TEST_DATASET_PATH = ROOT_DATASET_PATH / 'test'
DEVICE = "cpu"

In [3]:
data = load_dataset('imagefolder', data_dir="./cifake/")

Resolving data files:   0%|          | 0/100000 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20000 [00:00<?, ?it/s]

Found cached dataset imagefolder (/home/sabiyyuhakim/.cache/huggingface/datasets/imagefolder/default-b49058962a1a0797/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [5]:
checkpoint = "microsoft/resnet-152"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

image_processor

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [6]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean,
                      std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
    [
        RandomResizedCrop(crop_size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(crop_size),
        ToTensor(),
        normalize,
    ]
)

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    example_batch["pixel_values"] = [val_transforms(
        image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

train_ds = data["train"]
test_ds = data["test"]

train_ds.set_transform(preprocess_train)
test_ds.set_transform(preprocess_val)

In [7]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

model.to(DEVICE)

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-152 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([2, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

/tmp/ipykernel_602487/3843272848.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [9]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"]
                               for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [10]:
batch_size = 32
gradient_accumulation_steps = 4 
model_name = checkpoint.split("/")[-1]
epochs = 5
learning_rate = 5e-5

In [11]:
train_loader = DataLoader(
    train_ds, batch_size=batch_size, collate_fn=collate_fn)
eval_loader = DataLoader(test_ds, batch_size=batch_size, collate_fn=collate_fn)


optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=len(train_loader) * epochs * 0.1,
    num_training_steps=len(train_loader) * epochs
)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
import wandb
wandb.login()
wandb.init(project="proyek-akhir-pcd", name="resnet152-5epoch")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sulthanabiyyu. Use `wandb login --relogin` to force relogin


In [19]:
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for batch_num, batch in enumerate(train_loader):
        inputs = batch["pixel_values"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels) / gradient_accumulation_steps
        loss.backward()

        if (batch_num + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item()

    # Calculate average training loss for this epoch
    avg_train_loss = total_loss / len(train_loader)

    # Evaluation loop
    model.eval()
    total_eval_loss = 0.0

    if batch_num % 100 == 0:
        
        with torch.no_grad():
            f1_eval = 0.0
            for batch in eval_loader:
                inputs = batch["pixel_values"].to(DEVICE)
                labels = batch["labels"].to(DEVICE)

                outputs = model(inputs)

                # Define your loss function here
                loss = loss_fn(outputs.logits, labels)
                f1_eval += compute_metrics(outputs)
                
                total_eval_loss += loss.item()
            f1_eval = f1_eval / len(eval_loader)

        # Calculate average evaluation loss for this epoch
        avg_eval_loss = total_eval_loss / len(eval_loader)

        # Record all statistics from this step
        wandb.log({
            "epoch": epoch,
            "step": batch_num,
            "train_loss": avg_train_loss,
            "eval_loss": avg_eval_loss,
            "f1_eval": f1_eval
        })
        torch.save(model.state_dict(),
           f"{model_name}-finetuned-cifake-epoch{epochs}-steps{batch_num}.pt")
    
    
    # Print training and evaluation loss for this epoch
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Eval Loss: {avg_eval_loss:.4f}")

KeyboardInterrupt: 

In [18]:
outputs

ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[ 0.0187, -0.0776],
        [-0.0700, -0.0595],
        [ 0.0088, -0.1338],
        [-0.0395, -0.1108],
        [ 0.0066, -0.0151],
        [-0.0083, -0.1446],
        [-0.0311, -0.0540],
        [-0.0850, -0.0187],
        [-0.0812, -0.0747],
        [ 0.0122,  0.0112],
        [-0.0215, -0.0283],
        [ 0.0754, -0.1103],
        [-0.1283, -0.0281],
        [-0.0267, -0.0722],
        [-0.0442, -0.0081],
        [-0.0358, -0.0338],
        [-0.0681, -0.0308],
        [-0.0060, -0.0363],
        [-0.1562, -0.1841],
        [-0.1275,  0.0019],
        [-0.0523, -0.0142],
        [-0.0718, -0.0133],
        [-0.0899, -0.0471],
        [-0.0397, -0.0862],
        [ 0.0446, -0.0944],
        [-0.0292, -0.0209],
        [-0.0135, -0.0494],
        [ 0.0238, -0.0529],
        [-0.0178, -0.0748],
        [ 0.0671, -0.0943],
        [-0.0372, -0.0586],
        [-0.0654, -0.1033]], grad_fn=<AddmmBackward0>), hidden_states=None)